In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyreadstat
import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

In [2]:
# calling biomarkers
bloodbiomk1_file_path = r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers\bloodbiomk1_20231117.csv"
bloodbiomk1 = pd.read_csv(bloodbiomk1_file_path)
bloodbiomk1 = pd.DataFrame(bloodbiomk1)

bloodbiomk2_file_path = r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers\bloodbiomk2_20231117.csv"
bloodbiomk2 = pd.read_csv(bloodbiomk2_file_path)
bloodbiomk2 = pd.DataFrame(bloodbiomk2)

csfbiomk1_file_path = r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers\csfbiomk1_20231117.csv"
csfbiomk1 = pd.read_csv(csfbiomk1_file_path)
csfbiomk1 = pd.DataFrame(csfbiomk1)

csfbiomk2_file_path = r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers\csfbiomk2_20231117.csv"
csfbiomk2 = pd.read_csv(csfbiomk2_file_path)
csfbiomk2 = pd.DataFrame(csfbiomk2)

csfbiomk3_file_path = r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers\csfbiomk3_20231117.csv"
csfbiomk3 = pd.read_csv(csfbiomk3_file_path)
csfbiomk3 = pd.DataFrame(csfbiomk3)



In [3]:
import os
main_path = r'Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers'
folder_name = 'blood_biomk1_to_briefa'  
save_path = os.path.join(main_path, folder_name)
os.makedirs(save_path, exist_ok=True)

# save_path = r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers\blood_biomk1_briefa" # Update this path to your desired folder


In [4]:
# Check for null values in each column
bloodbiomk2_nulls = bloodbiomk2.isnull().sum()

bloodbiomk2_nulls_df = pd.DataFrame({
    'Column': bloodbiomk2_nulls.index,
    'Number of Nulls': bloodbiomk2_nulls.values
})

bloodbiomk2_nulls_df_transposed = bloodbiomk2_nulls_df.T

print("Null values in bloodbiomk2:")
print(bloodbiomk2_nulls_df)
bloodbiomk2_nulls_df_transposed.to_csv(r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers\bloodbiomk2_nulls.csv", index=False)


Null values in bloodbiomk2:
                   Column  Number of Nulls
0              subject_id                0
1                   visit                0
2             checkin_bin                0
3             exposurebin                0
4              age_decade                0
..                    ...              ...
127              p_VEGF_A                4
128             p_asyn_cv                4
129                p_asyn                4
130                  p_Hb                4
131  dxcte_asyn_p_flag_dv                4

[132 rows x 2 columns]


In [5]:
# Calculate the mean for each column, ignoring NaNs
# Replace nulls with label means

for column in bloodbiomk2.columns:
    if column != 'checkin_bin':  # Skip the label column

        averages = bloodbiomk2.groupby('checkin_bin')[column].mean()
        
        # Replace nulls for each label separately
        for label in averages.index:
            bloodbiomk2.loc[(bloodbiomk2['checkin_bin'] == label) & (bloodbiomk2[column].isnull()), column] = averages[label]

print(bloodbiomk2.head())

   subject_id  visit  checkin_bin  exposurebin  age_decade  racecat_combined  \
0      1001.0      1            2            1         1.0               5.0   
1      1002.0      1            2            1         1.0               5.0   
2      1003.0      1            2            1         1.0               5.0   
3      1004.0      1            1            1         2.0               5.0   
4      1005.0      1            3            0         2.0               5.0   

   eduyears  totyr_foot  chiiseas_pf  chiiyrs_pf  ...  p_IL_7_cv  p_IL_7  \
0      16.0         7.0       4335.4      2167.7  ...      3.060    3.06   
1      15.0        14.0      10363.1      5708.1  ...      2.440   12.30   
2      18.0        12.0       6685.4      4863.9  ...      3.960    1.87   
3      16.0        16.0       7701.2      6448.9  ...      5.940    9.81   
4      21.0         NaN          NaN         NaN  ...      0.428    8.12   

   p_TNF_beta_cv  p_TNF_beta  p_VEGF_A_cv  p_VEGF_A  p_asyn_cv

In [6]:
bloodbiomk2.to_csv(r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers\bloodbiomk2_nullreplaced.csv")
original_data = bloodbiomk2.copy()

In [7]:
bloodbiomk2 = bloodbiomk2.drop(columns=bloodbiomk2.loc[:,'subject_id':'chiiyrs_pg'].columns)
# bloodbiomk2 = bloodbiomk2.drop(columns='p_PDGFRbeta_FLAG_other') # This is not for this dataset unless you find a column where there are too many nulls

bloodbiomk2.head()

,Qalb,s_alb,p_IL_17A_F_cv,p_IL_17A_F,p_IL_17B_cv,p_IL_17B,p_IL_17C_cv,p_IL_17C,p_IL_17D_cv,p_IL_17D,...,p_IL_7_cv,p_IL_7,p_TNF_beta_cv,p_TNF_beta,p_VEGF_A_cv,p_VEGF_A,p_asyn_cv,p_asyn,p_Hb,dxcte_asyn_p_flag_dv
0,7.852013,3.912500,5.43,2.190,1.63,1.18,7.39,1.59,14.40,28.40,...,3.060,3.06,6.040,0.1210,0.943,16.6,0.51,178000.0,37538.80,0.0
1,6.994813,3.556088,6.84,3.460,4.52,2.30,3.30,2.23,8.79,34.30,...,2.440,12.30,0.331,0.2850,2.640,55.8,12.60,170000.0,26355.70,0.0
2,6.994813,3.556088,2.49,0.126,2.63,2.30,7.44,17.50,27.00,9.01,...,3.960,1.87,15.400,0.8180,5.680,25.3,0.14,79500.0,148871.00,0.0
3,7.867197,3.414635,1.05,8.280,5.71,2.95,9.27,1.94,6.47,35.30,...,5.940,9.81,1.860,0.0472,0.192,62.5,5.33,117000.0,20893.80,0.0
4,7.429948,3.378848,18.90,1.180,2.58,1.16,3.81,6.84,26.20,26.10,...,0.428,8.12,0.306,0.1560,5.370,99.2,4.54,101000.0,8102.65,0.0


In [8]:
bloodbiomk2.head(10)

,Qalb,s_alb,p_IL_17A_F_cv,p_IL_17A_F,p_IL_17B_cv,p_IL_17B,p_IL_17C_cv,p_IL_17C,p_IL_17D_cv,p_IL_17D,...,p_IL_7_cv,p_IL_7,p_TNF_beta_cv,p_TNF_beta,p_VEGF_A_cv,p_VEGF_A,p_asyn_cv,p_asyn,p_Hb,dxcte_asyn_p_flag_dv
0,7.852013,3.912500,5.43,2.1900,1.630,1.180,7.39,1.590,14.4000,28.40,...,3.060,3.06,6.040,0.1210,0.943,16.6,0.51,178000.0,37538.80,0.0
1,6.994813,3.556088,6.84,3.4600,4.520,2.300,3.30,2.230,8.7900,34.30,...,2.440,12.30,0.331,0.2850,2.640,55.8,12.60,170000.0,26355.70,0.0
2,6.994813,3.556088,2.49,0.1260,2.630,2.300,7.44,17.500,27.0000,9.01,...,3.960,1.87,15.400,0.8180,5.680,25.3,0.14,79500.0,148871.00,0.0
3,7.867197,3.414635,1.05,8.2800,5.710,2.950,9.27,1.940,6.4700,35.30,...,5.940,9.81,1.860,0.0472,0.192,62.5,5.33,117000.0,20893.80,0.0
4,7.429948,3.378848,18.90,1.1800,2.580,1.160,3.81,6.840,26.2000,26.10,...,0.428,8.12,0.306,0.1560,5.370,99.2,4.54,101000.0,8102.65,0.0
5,6.994813,3.556088,11.60,0.2020,5.270,5.490,1.08,1.400,4.5800,15.50,...,0.532,1.83,23.200,0.3190,1.380,17.0,3.96,137000.0,1255800.00,0.0
6,7.867197,3.414635,8.85,0.1720,0.777,0.592,2.35,1.440,0.0901,16.80,...,3.280,2.26,9.930,0.2110,1.360,17.6,1.88,72800.0,185866.00,0.0
7,8.743853,3.111100,2.52,0.0754,1.470,1.120,0.00,3.470,3.5400,40.70,...,0.343,2.97,77.400,0.9020,2.570,22.3,3.92,101000.0,117826.00,0.0
8,7.867197,3.414635,4.80,4.1500,5.460,0.824,6.05,4.780,44.4000,25.30,...,7.320,2.49,4.450,0.1380,0.976,60.3,1.65,145000.0,412702.00,1.0
9,6.994813,3.556088,5.43,0.1870,1.240,0.797,1.71,0.999,35.4000,34.70,...,1.220,1.53,5.350,0.3840,3.570,16.5,4.34,47900.0,15836.20,0.0


In [9]:
# Calculate the mean for each column, ignoring NaNs
mean_values = bloodbiomk2.mean()

bloodbiomk2 = bloodbiomk2.fillna(mean_values)

bloodbiomk2.head()

,Qalb,s_alb,p_IL_17A_F_cv,p_IL_17A_F,p_IL_17B_cv,p_IL_17B,p_IL_17C_cv,p_IL_17C,p_IL_17D_cv,p_IL_17D,...,p_IL_7_cv,p_IL_7,p_TNF_beta_cv,p_TNF_beta,p_VEGF_A_cv,p_VEGF_A,p_asyn_cv,p_asyn,p_Hb,dxcte_asyn_p_flag_dv
0,7.852013,3.912500,5.43,2.190,1.63,1.18,7.39,1.59,14.40,28.40,...,3.060,3.06,6.040,0.1210,0.943,16.6,0.51,178000.0,37538.80,0.0
1,6.994813,3.556088,6.84,3.460,4.52,2.30,3.30,2.23,8.79,34.30,...,2.440,12.30,0.331,0.2850,2.640,55.8,12.60,170000.0,26355.70,0.0
2,6.994813,3.556088,2.49,0.126,2.63,2.30,7.44,17.50,27.00,9.01,...,3.960,1.87,15.400,0.8180,5.680,25.3,0.14,79500.0,148871.00,0.0
3,7.867197,3.414635,1.05,8.280,5.71,2.95,9.27,1.94,6.47,35.30,...,5.940,9.81,1.860,0.0472,0.192,62.5,5.33,117000.0,20893.80,0.0
4,7.429948,3.378848,18.90,1.180,2.58,1.16,3.81,6.84,26.20,26.10,...,0.428,8.12,0.306,0.1560,5.370,99.2,4.54,101000.0,8102.65,0.0


In [10]:
# calling Neuropsychiatric Measurements

briefa_file_path = r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers\briefa_20231117.csv"
briefa = pd.read_csv(briefa_file_path)
briefa = pd.DataFrame(briefa)


In [11]:
# Check for null values in each column
briefa_nulls = briefa.isnull().sum()

briefa_nulls_df = pd.DataFrame({
    'Column': briefa_nulls.index,
    'Number of Nulls': briefa_nulls.values
})

briefa_nulls_df_transposed = briefa_nulls_df.T

print("Null values in briefa:")
print(briefa_nulls_df)
briefa_nulls_df_transposed.to_csv(r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers\briefa_nulls.csv", index=False)


Null values in briefa:
          Column  Number of Nulls
0     subject_id                0
1          visit                0
2    checkin_bin                0
3    exposurebin                0
4     age_decade                0
..           ...              ...
102      workorg                0
103    upseteasy                0
104    impulsive                0
105       pickup                0
106     complete                0

[107 rows x 2 columns]


In [12]:
briefa.head()

,subject_id,visit,checkin_bin,exposurebin,age_decade,racecat_combined,eduyears,totyr_foot,chiiseas_pf,chiiyrs_pf,...,activityorg,getover,onething,moodchange,consequence,workorg,upseteasy,impulsive,pickup,complete
0,1001,1,2,1,1,5,16.0,7.0,4335.4,2167.7,...,2,1,2,1,1,2,2,1,1,2
1,1002,1,2,1,1,5,15.0,14.0,10363.1,5708.1,...,3,3,3,2,3,3,3,3,2,3
2,1003,1,2,1,1,5,18.0,12.0,6685.4,4863.9,...,3,3,3,3,2,3,3,3,1,2
3,1004,1,1,1,2,5,16.0,16.0,7701.2,6448.9,...,1,1,1,1,1,1,2,1,1,1
4,1005,1,3,0,2,5,21.0,NaN,NaN,NaN,...,1,2,1,1,1,1,1,1,1,1


In [13]:
briefa = briefa.drop(columns=briefa.loc[:,'subject_id':'chiiyrs_pg'].columns)
briefa = briefa.drop(columns=briefa.loc[:,'negativ':'complete'].columns)
briefa.head()

,mi,tmi,gec,tgec,bri,tbri,inhibit,shift,emotcont,selfmon,initiate,workmem,planorg,taskmon,orgmat
0,75,64,114,57,39,46,12,9,11,7,14,17,20,13,11
1,101,85,171,85,70,79,17,15,23,15,19,24,27,15,16
2,90,77,158,79,68,77,15,14,27,12,20,22,22,14,12
3,43,40,81,42,38,46,9,8,15,6,9,8,11,7,8
4,43,39,81,41,38,45,9,11,12,6,10,8,10,7,8


In [14]:
briefa.describe()

,mi,tmi,gec,tgec,bri,tbri,inhibit,shift,emotcont,selfmon,initiate,workmem,planorg,taskmon,orgmat
count,236.000000,236.000000,236.000000,236.000000,236.000000,236.000000,236.000000,236.000000,236.000000,236.000000,236.000000,236.000000,236.000000,236.000000,236.000000
mean,63.682203,56.724576,110.169492,56.508475,46.487288,55.203390,12.169492,9.690678,15.758475,8.868644,12.605932,13.758475,15.338983,9.466102,12.512712
std,19.879120,16.771696,33.473344,17.217382,14.657067,16.438698,3.606861,3.378543,5.769042,3.063115,4.101039,4.922909,5.199379,2.946855,4.335387
min,40.000000,36.000000,70.000000,35.000000,30.000000,36.000000,8.000000,6.000000,10.000000,6.000000,8.000000,8.000000,10.000000,6.000000,8.000000
25%,45.000000,41.000000,79.750000,41.000000,34.000000,41.000000,9.000000,6.000000,10.000000,6.000000,9.000000,9.000000,10.000000,6.000000,9.000000
50%,60.000000,53.000000,104.500000,54.000000,43.000000,51.000000,11.000000,9.000000,14.000000,8.000000,12.000000,13.000000,14.000000,9.000000,11.000000
75%,79.250000,68.250000,136.000000,69.000000,57.000000,67.250000,14.000000,12.000000,20.000000,11.000000,16.000000,18.000000,19.250000,12.000000,15.000000
max,116.000000,104.000000,204.000000,108.000000,88.000000,105.000000,23.000000,18.000000,30.000000,18.000000,23.000000,24.000000,29.000000,18.000000,24.000000


In [15]:
bloodbiomk2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 236 entries, 0 to 235
Columns: 118 entries, Qalb to dxcte_asyn_p_flag_dv
dtypes: float64(118)
memory usage: 217.7 KB


In [16]:
import os
main_path = r'Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers'
folder_name = 'blood_biomk1_to_briefa'  
save_path = os.path.join(main_path, folder_name)
os.makedirs(save_path, exist_ok=True)


In [17]:
categories_file_path = r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers\categories.csv"
categories_df = pd.read_csv(categories_file_path)
new_column = categories_df['checkin_bin']
briefa['category'] = new_column
print(briefa.head())

bloodbiomk2['category'] = new_column
print(bloodbiomk2.head())


    mi  tmi  gec  tgec  bri  tbri  inhibit  shift  emotcont  selfmon  \
0   75   64  114    57   39    46       12      9        11        7   
1  101   85  171    85   70    79       17     15        23       15   
2   90   77  158    79   68    77       15     14        27       12   
3   43   40   81    42   38    46        9      8        15        6   
4   43   39   81    41   38    45        9     11        12        6   

   initiate  workmem  planorg  taskmon  orgmat  category  
0        14       17       20       13      11         2  
1        19       24       27       15      16         2  
2        20       22       22       14      12         2  
3         9        8       11        7       8         1  
4        10        8       10        7       8         3  
       Qalb     s_alb  p_IL_17A_F_cv  p_IL_17A_F  p_IL_17B_cv  p_IL_17B  \
0  7.852013  3.912500           5.43       2.190         1.63      1.18   
1  6.994813  3.556088           6.84       3.460         4.52  

C:\Users\fargor\AppData\Local\Temp\ipykernel_48664\2199802666.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  bloodbiomk2['category'] = new_column


In [18]:
# bloodbiomk2['category'] = briefa['category']

# correlation_matrix = bloodbiomk2.corr()

# plt.figure(figsize=(12, 8))
# sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', square=True)
# plt.title('Correlation Heatmap of Features')
# plt.show()

# # Alternatively, scatter plots for each feature vs y (checkin_bin)
# for feature in bloodbiomk2.columns[:-1]:  # Exclude the label column
#     plt.figure(figsize=(10, 6))
#     sns.boxplot(x='category', y=feature, data=bloodbiomk2)
#     plt.title(f'Boxplot of {feature} by category')
#     plt.xlabel('category')
#     plt.ylabel(feature)
#     plt.show()


In [19]:
bloodbiomk2.copy()

,Qalb,s_alb,p_IL_17A_F_cv,p_IL_17A_F,p_IL_17B_cv,p_IL_17B,p_IL_17C_cv,p_IL_17C,p_IL_17D_cv,p_IL_17D,...,p_IL_7,p_TNF_beta_cv,p_TNF_beta,p_VEGF_A_cv,p_VEGF_A,p_asyn_cv,p_asyn,p_Hb,dxcte_asyn_p_flag_dv,category
0,7.852013,3.912500,5.430,2.1900,1.63,1.180,7.39,1.590,14.40,28.40,...,3.06,6.040,0.1210,0.943,16.6,0.51,178000.0,37538.80,0.0,2
1,6.994813,3.556088,6.840,3.4600,4.52,2.300,3.30,2.230,8.79,34.30,...,12.30,0.331,0.2850,2.640,55.8,12.60,170000.0,26355.70,0.0,2
2,6.994813,3.556088,2.490,0.1260,2.63,2.300,7.44,17.500,27.00,9.01,...,1.87,15.400,0.8180,5.680,25.3,0.14,79500.0,148871.00,0.0,2
3,7.867197,3.414635,1.050,8.2800,5.71,2.950,9.27,1.940,6.47,35.30,...,9.81,1.860,0.0472,0.192,62.5,5.33,117000.0,20893.80,0.0,1
4,7.429948,3.378848,18.900,1.1800,2.58,1.160,3.81,6.840,26.20,26.10,...,8.12,0.306,0.1560,5.370,99.2,4.54,101000.0,8102.65,0.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231,3.776207,3.367400,6.800,1.0200,9.12,0.613,2.00,1.420,4.99,46.40,...,1.66,3.820,0.1430,0.105,17.2,2.40,32100.0,48562.30,0.0,2
232,7.384902,3.279800,1.890,0.0806,36.90,0.527,25.10,0.715,70.20,19.90,...,6.05,1.730,0.0330,0.540,31.8,4.28,147000.0,112602.00,0.0,3
233,4.568572,3.771200,0.000,0.3330,2.01,1.490,7.64,1.600,10.10,26.40,...,0.71,0.000,0.1340,3.290,9.2,7.49,33000.0,6892.03,0.0,2
234,11.101832,2.897900,0.433,0.1210,1.72,2.200,1.64,3.910,1.04,25.00,...,2.60,7.930,0.1040,0.492,68.5,3.45,42900.0,30735.40,0.0,1



# 95th percentile for both analysis and plotting

In [20]:
for feature in bloodbiomk2.columns[:-1]:  # Assuming 'category' is the last column
    upper_limit = bloodbiomk2[feature].quantile(0.95)  # Cap at 95th percentile
    lower_limit = bloodbiomk2[feature].quantile(0.05)  # Floor at 5th percentile
    bloodbiomk2[feature] = np.where(bloodbiomk2[feature] > upper_limit, upper_limit, bloodbiomk2[feature])
    bloodbiomk2[feature] = np.where(bloodbiomk2[feature] < lower_limit, lower_limit, bloodbiomk2[feature])

    

# Initial Step for Trainings

In [21]:
X = bloodbiomk2.copy()

In [22]:
X

,Qalb,s_alb,p_IL_17A_F_cv,p_IL_17A_F,p_IL_17B_cv,p_IL_17B,p_IL_17C_cv,p_IL_17C,p_IL_17D_cv,p_IL_17D,...,p_IL_7,p_TNF_beta_cv,p_TNF_beta,p_VEGF_A_cv,p_VEGF_A,p_asyn_cv,p_asyn,p_Hb,dxcte_asyn_p_flag_dv,category
0,7.852013,3.912500,5.430,2.1900,1.63,1.180,7.3900,1.5900,14.40,28.400,...,3.0600,6.04000,0.121000,0.943,16.6,0.51000,178000.0,37538.80,0.0,2
1,6.994813,3.556088,6.840,3.4600,4.52,2.300,3.3000,2.2300,8.79,34.300,...,9.5525,0.33100,0.285000,2.640,55.8,7.23750,170000.0,26355.70,0.0,2
2,6.994813,3.556088,2.490,0.1260,2.63,2.300,7.4400,7.2650,27.00,9.010,...,1.8700,15.40000,0.671500,5.680,25.3,0.21875,79500.0,148871.00,0.0,2
3,7.867197,3.414635,1.050,5.0025,5.71,2.950,9.2700,1.9400,6.47,35.300,...,9.5525,1.86000,0.047200,0.192,62.5,5.33000,117000.0,20893.80,0.0,1
4,7.429948,3.378848,14.025,1.1800,2.58,1.160,3.8100,6.8400,26.20,26.100,...,8.1200,0.32475,0.156000,5.370,67.9,4.54000,101000.0,8102.65,0.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231,4.213896,3.367400,6.800,1.0200,9.12,0.613,2.0000,1.4200,4.99,40.325,...,1.6600,3.82000,0.143000,0.140,17.2,2.40000,32100.0,48562.30,0.0,2
232,7.384902,3.279800,1.890,0.0806,9.32,0.527,9.4775,0.7165,33.95,19.900,...,6.0500,1.73000,0.039625,0.540,31.8,4.28000,147000.0,112602.00,0.0,3
233,4.568572,3.771200,0.218,0.3330,2.01,1.490,7.6400,1.6000,10.10,26.400,...,1.3400,0.32475,0.134000,3.290,13.6,7.23750,33000.0,6892.03,0.0,2
234,11.101832,2.926275,0.433,0.1210,1.72,2.200,1.6400,3.9100,1.04,25.000,...,2.6000,7.93000,0.104000,0.492,67.9,3.45000,42900.0,30735.40,0.0,1


In [23]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# got error due to non numeric values, so removing them:

# Assume bloodbiomk2 is your DataFrame loaded with various types of data
X = bloodbiomk2.copy()

# Select only numeric columns from the DataFrame
X_numeric = X.select_dtypes(include=[np.number])
X = X_numeric

# Now X_numeric contains only the numeric columns of the original DataFrame
# Proceed with scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split into training and testing sets
X_train, X_test = train_test_split(X_scaled, test_size=0.3, random_state=42)


models = {}
predictions = {}


X_train, X_test = train_test_split(X_scaled, test_size=0.3, random_state=42)

In [24]:
# Define the columns you want to select for modeling
## columns = ['mi', 'tmi','gec','tgec','bri','tbri','inhibit','shift','emotcont','selfmon','initiate','workmem','planorg','taskmon','orgmat'] 
selected_columns = ['orgmat']  # Replace 'column1' and 'column2' with the actual column names

# selected_columns = ['mi', 'tmi','gec','tgec','bri','tbri','inhibit','shift','emotcont','selfmon','initiate','workmem','planorg','taskmon','orgmat']


In [25]:
selected_columns

['orgmat']

In [26]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

# Initialize an empty list to store results
results = []
feature_importance_rankings = {}
model_name = "Random Forest"
selected_columns_str = "_".join(selected_columns)

# First, calculate feature importances for each target column and save rankings
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # Train full model to get feature importances
    model = RandomForestRegressor(random_state=42)
    model.fit(X_train, y_train)
    
    # Get feature importance and rank features
    feature_importance = pd.Series(model.feature_importances_, index=X_train.columns)
    ranked_features = feature_importance.sort_values(ascending=False)
    
    # Save the rankings in a dictionary
    feature_importance_rankings[column] = ranked_features
    
    # Save performance metrics with all features as a baseline
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_bloodbiomk2_rank_{selected_columns_str}.xlsx") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")


# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        
        # Select the top `n_features` based on initial ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate model with selected features
        model = RandomForestRegressor(random_state=42)
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Append results for each number of top features
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_bloodbiomk2_w_top_features_{selected_columns_str}.xlsx") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to feature_importance_rankings.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")



Feature importance rankings saved to feature_importance_rankings.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [27]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression

# Initialize an empty list to store results and a dictionary for feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "Multiple Linear Regression"
selected_columns_str = "_".join(selected_columns)

# Standardize features for linear regression
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# First, calculate feature importances using SelectKBest for each target column and save rankings
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
    
    # Feature selection using SelectKBest
    selector = SelectKBest(score_func=f_regression, k="all")
    selector.fit(X_train, y_train)
    feature_scores = pd.Series(selector.scores_, index=X_train.columns)
    ranked_features = feature_scores.sort_values(ascending=False)
    
    # Save the rankings in a dictionary
    feature_importance_rankings[column] = ranked_features
    
    # Train full model with all features as a baseline
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # Evaluate and store baseline performance metrics with all features
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_bloodbiomk2_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
        
        # Select the top `n_features` based on initial ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate model with selected features
        model = LinearRegression()
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Append results for each number of top features
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_bloodbiomk2_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [28]:
import pandas as pd
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression

# Initialize an empty list to store results and a dictionary for feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "Gradient Boosting Machines"
selected_columns_str = "_".join(selected_columns)

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# First, calculate feature importances using SelectKBest for each target column and save rankings
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
    
    # Feature selection using SelectKBest
    selector = SelectKBest(score_func=f_regression, k="all")
    selector.fit(X_train, y_train)
    feature_scores = pd.Series(selector.scores_, index=X_train.columns)
    ranked_features = feature_scores.sort_values(ascending=False)
    
    # Save the rankings in a dictionary
    feature_importance_rankings[column] = ranked_features
    
    # Train full model with all features as a baseline
    model = LGBMRegressor(random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # Evaluate and store baseline performance metrics with all features
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_bloodbiomk2_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
        
        # Select the top `n_features` based on initial ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate model with selected features
        model = LGBMRegressor(random_state=42)
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Append results for each number of top features
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_bloodbiomk2_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000716 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5854
[LightGBM] [Info] Number of data points in the train set: 165, number of used features: 119
[LightGBM] [Info] Start training from score 12.466667
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000146 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1412
[LightGBM] [Info] Number of data points in the train set: 165, number of used features: 29
[LightGBM] [Info] Start training from score 12.466667
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000593 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5105
[LightGBM] [Info] Number of data points in the train set: 165, number of used features: 103
[LightGBM] [Info] Start training from score 12.466667
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

[LightGBM] [Info] Total Bins 5361
[LightGBM] [Info] Number of data points in the train set: 165, number of used features: 109
[LightGBM] [Info] Start training from score 12.466667
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [29]:
import pandas as pd
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression

# Initialize an empty list to store results and a dictionary for feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "XGBoost"
selected_columns_str = "_".join(selected_columns)

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# First, calculate feature importances using SelectKBest for each target column and save rankings
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
    
    # Feature selection using SelectKBest
    selector = SelectKBest(score_func=f_regression, k="all")
    selector.fit(X_train, y_train)
    feature_scores = pd.Series(selector.scores_, index=X_train.columns)
    ranked_features = feature_scores.sort_values(ascending=False)
    
    # Save the rankings in a dictionary
    feature_importance_rankings[column] = ranked_features
    
    # Train full model with all features as a baseline
    model = xgb.XGBRegressor(random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # Evaluate and store baseline performance metrics with all features
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_bloodbiomk2_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
        
        # Select the top `n_features` based on initial ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate model with selected features
        model = xgb.XGBRegressor(random_state=42)
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Append results for each number of top features
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_bloodbiomk2_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [30]:
import pandas as pd
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression

# Initialize an empty list to store results and a dictionary for feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "Support Vector Regression"
selected_columns_str = "_".join(selected_columns)

# Standardize features for SVR
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# Calculate feature importances using SelectKBest for each target column and save rankings
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
    
    # Feature selection using SelectKBest
    selector = SelectKBest(score_func=f_regression, k="all")
    selector.fit(X_train, y_train)
    feature_scores = pd.Series(selector.scores_, index=X_train.columns)
    ranked_features = feature_scores.sort_values(ascending=False)
    
    # Save the rankings in a dictionary
    feature_importance_rankings[column] = ranked_features
    
    # Train SVR model with all features as a baseline
    model = SVR(kernel='rbf', C=0.2, epsilon=0.01)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # Evaluate and store baseline performance metrics with all features
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_bloodbiomk2_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
        
        # Select the top `n_features` based on initial ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate model with selected features
        model = SVR(kernel='rbf', C=0.2, epsilon=0.01)
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Append results for each number of top features
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_bloodbiomk2_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [31]:
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Initialize an empty list to store results and a dictionary for feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "ANN"
selected_columns_str = "_".join(selected_columns)

# Standardize features for ANN
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# Feature selection and baseline model training
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
    
    # Feature selection using SelectKBest
    selector = SelectKBest(score_func=f_regression, k="all")
    selector.fit(X_train, y_train)
    feature_scores = pd.Series(selector.scores_, index=X_train.columns)
    ranked_features = feature_scores.sort_values(ascending=False)
    
    # Save the rankings in a dictionary
    feature_importance_rankings[column] = ranked_features
    
    # Train ANN model with all features as a baseline
    X_train_selected = selector.transform(X_train)
    X_test_selected = selector.transform(X_test)
    
    model = Sequential([
        Dense(128, activation='relu', input_dim=X_train_selected.shape[1]),
        Dense(64, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train_selected, y_train, epochs=100, batch_size=10, verbose=0)
    
    y_pred = model.predict(X_test_selected).flatten()
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_bloodbiomk2_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
        
        # Select the top `n_features` based on initial ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Scale selected features for ANN
        X_train_selected_scaled = scaler.fit_transform(X_train_selected)
        X_test_selected_scaled = scaler.transform(X_test_selected)
        
        # Train and evaluate ANN model with selected features
        model = Sequential([
            Dense(128, activation='relu', input_dim=X_train_selected_scaled.shape[1]),
            Dense(64, activation='relu'),
            Dense(1)
        ])
        model.compile(optimizer='adam', loss='mse')
        model.fit(X_train_selected_scaled, y_train, epochs=100, batch_size=10, verbose=0)
        
        y_pred = model.predict(X_test_selected_scaled).flatten()
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Append results for each number of top features
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_bloodbiomk2_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


3/3 [==============================] - 0s 1ms/step


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


3/3 [==============================] - 0s 997us/step
Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [32]:
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression

# Initialize an empty list to store results and a dictionary for feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "KNeighbors Regressor"
selected_columns_str = "_".join(selected_columns)

# Standardize features for KNN
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# Feature selection and baseline model training
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
    
    # Feature selection using SelectKBest
    selector = SelectKBest(score_func=f_regression, k="all")
    selector.fit(X_train, y_train)
    feature_scores = pd.Series(selector.scores_, index=X_train.columns)
    ranked_features = feature_scores.sort_values(ascending=False)
    
    # Save the rankings in a dictionary
    feature_importance_rankings[column] = ranked_features
    
    # Train KNN model with all features as a baseline
    X_train_selected = selector.transform(X_train)
    X_test_selected = selector.transform(X_test)
    
    model = KNeighborsRegressor(n_neighbors=5)
    model.fit(X_train_selected, y_train)
    
    y_pred = model.predict(X_test_selected)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_bloodbiomk2_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
        
        # Select the top `n_features` based on initial ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate KNN model with selected features
        model = KNeighborsRegressor(n_neighbors=5)
        model.fit(X_train_selected, y_train)
        
        y_pred = model.predict(X_test_selected)
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Append results for each number of top features
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_bloodbiomk2_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [33]:
import pandas as pd
from sklearn.ensemble import StackingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_regression

# Initialize an empty list to store results and a dictionary for feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "Stacking Regressor"
selected_columns_str = "_".join(selected_columns)

# Define the base models and final estimator for stacking
base_models = [
    ('knn', KNeighborsRegressor(n_neighbors=13, leaf_size=23, p=1, weights='uniform')),
    ('svr', SVR(kernel='rbf', C=0.2, epsilon=0.01))
]
final_estimator = Ridge()
stack_model = StackingRegressor(estimators=base_models, final_estimator=final_estimator, cv=5)

# Feature selection and baseline model training
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # Feature selection using SelectKBest
    selector = SelectKBest(score_func=f_regression, k="all")
    selector.fit(X_train, y_train)
    feature_scores = pd.Series(selector.scores_, index=X_train.columns)
    ranked_features = feature_scores.sort_values(ascending=False)
    
    # Save the rankings in a dictionary
    feature_importance_rankings[column] = ranked_features
    
    # Train stacking model with all features as a baseline
    X_train_selected = selector.transform(X_train)
    X_test_selected = selector.transform(X_test)
    
    stack_model.fit(X_train_selected, y_train)
    y_pred = stack_model.predict(X_test_selected)
    
    # Evaluate and store baseline performance metrics with all features
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_bloodbiomk2_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        
        # Select the top `n_features` based on initial ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate stacking model with selected features
        stack_model.fit(X_train_selected, y_train)
        y_pred = stack_model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Append results for each number of top features
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_bloodbiomk2_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [34]:
import pandas as pd
from sklearn.svm import SVR
from sklearn.ensemble import BaggingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_regression

# Initialize an empty list to store results and a dictionary for feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "Bagging with SVR"
selected_columns_str = "_".join(selected_columns)

# Define the base model for Bagging
base_model = SVR(kernel='rbf', C=0.2, epsilon=0.01)

# Feature selection and baseline model training
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # Feature selection using SelectKBest
    selector = SelectKBest(score_func=f_regression, k="all")
    selector.fit(X_train, y_train)
    feature_scores = pd.Series(selector.scores_, index=X_train.columns)
    ranked_features = feature_scores.sort_values(ascending=False)
    
    # Save the rankings in a dictionary
    feature_importance_rankings[column] = ranked_features
    
    # Train Bagging model with all features as a baseline
    X_train_selected = selector.transform(X_train)
    X_test_selected = selector.transform(X_test)
    
    bagging_model = BaggingRegressor(base_estimator=base_model, n_estimators=10, random_state=42, n_jobs=-1)
    bagging_model.fit(X_train_selected, y_train)
    y_pred = bagging_model.predict(X_test_selected)
    
    # Evaluate and store baseline performance metrics with all features
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_bloodbiomk2_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        
        # Select the top `n_features` based on initial ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate Bagging model with selected features
        bagging_model = BaggingRegressor(base_estimator=base_model, n_estimators=10, random_state=42, n_jobs=-1)
        bagging_model.fit(X_train_selected, y_train)
        y_pred = bagging_model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Append results for each number of top features
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_bloodbiomk2_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:1

C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and w

C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and w

Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [35]:
import pandas as pd
from sklearn.svm import SVR
from sklearn.ensemble import BaggingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_regression

# Initialize an empty list to store results and a dictionary for feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "Bagging with SVR"
selected_columns_str = "_".join(selected_columns)

# Define the base model for Bagging
base_model = SVR(kernel='rbf', C=0.2, epsilon=0.01)

# Feature selection and baseline model training
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # Feature selection using SelectKBest
    selector = SelectKBest(score_func=f_regression, k="all")
    selector.fit(X_train, y_train)
    feature_scores = pd.Series(selector.scores_, index=X_train.columns)
    ranked_features = feature_scores.sort_values(ascending=False)
    
    # Save the rankings in a dictionary
    feature_importance_rankings[column] = ranked_features
    
    # Train Bagging model with all features as a baseline
    X_train_selected = selector.transform(X_train)
    X_test_selected = selector.transform(X_test)
    
    bagging_model = BaggingRegressor(base_estimator=base_model, n_estimators=10, random_state=42, n_jobs=-1)
    bagging_model.fit(X_train_selected, y_train)
    y_pred = bagging_model.predict(X_test_selected)
    
    # Evaluate and store baseline performance metrics with all features
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_bloodbiomk2_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        
        # Select the top `n_features` based on initial ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate Bagging model with selected features
        bagging_model = BaggingRegressor(base_estimator=base_model, n_estimators=10, random_state=42, n_jobs=-1)
        bagging_model.fit(X_train_selected, y_train)
        y_pred = bagging_model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Append results for each number of top features
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_bloodbiomk2_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:1

C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and w

C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and w

Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [36]:
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel

# Initialize an empty list to store results and a dictionary for feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "Lasso Regression"
selected_columns_str = "_".join(selected_columns)

# Standardize features for Lasso regression
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# Feature ranking and baseline model training
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
    
    # Lasso Regression Model with L1 regularization for feature selection
    model = Lasso(alpha=0.1)
    model.fit(X_train, y_train)
    
    # Select features based on non-zero coefficients
    selector = SelectFromModel(model, prefit=True)
    selected_features = X_train.columns[selector.get_support()]
    
    # Feature importance ranking based on absolute Lasso coefficients
    feature_importance = pd.Series(abs(model.coef_), index=X_train.columns)
    ranked_features = feature_importance.sort_values(ascending=False)
    feature_importance_rankings[column] = ranked_features

    # Evaluate baseline performance with all features
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_bloodbiomk2_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
        
        # Select top `n_features` based on ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate model with selected features
        model = Lasso(alpha=0.1)
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_bloodbiomk2_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [37]:
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel

# Initialize lists and dictionaries to store results and feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "Ridge Regression"
selected_columns_str = "_".join(selected_columns)

# Standardize features for Ridge regression
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# Feature ranking and baseline model training
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
    
    # Ridge Regression Model
    model = Ridge(alpha=1.0)
    model.fit(X_train, y_train)
    
    # Use coefficients for feature importance
    feature_importance = pd.Series(abs(model.coef_), index=X_train.columns)
    ranked_features = feature_importance.sort_values(ascending=False)
    feature_importance_rankings[column] = ranked_features
    
    # Evaluate baseline performance with all features
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_bloodbiomk2_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
        
        # Select top `n_features` based on ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate model with selected features
        model = Ridge(alpha=1.0)
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_bloodbiomk2_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [38]:
import pandas as pd
from sklearn.linear_model import BayesianRidge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Initialize lists and dictionaries for storing results and feature rankings
results = []
feature_importance_rankings = {}
model_name = "Bayesian Ridge"
selected_columns_str = "_".join(selected_columns)

# Standardize features for consistency
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# Feature ranking and baseline model training
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
    
    # Train full model to get feature importances
    model = BayesianRidge()
    model.fit(X_train, y_train)
    
    # Use coefficients for feature importance ranking
    feature_importance = pd.Series(abs(model.coef_), index=X_train.columns)
    ranked_features = feature_importance.sort_values(ascending=False)
    feature_importance_rankings[column] = ranked_features
    
    # Baseline performance with all features
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_bloodbiomk2_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
        
        # Select top `n_features` based on ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate model with selected features
        model = BayesianRidge()
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_bloodbiomk2_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [39]:
import pandas as pd
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression

# Initialize an empty list to store results and a dictionary for feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "Gaussian Process Regression"
selected_columns_str = "_".join(selected_columns)

# Define the kernel
kernel = RBF(1.0)

# Standardize features for Gaussian Process Regression
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# First, calculate feature importances using SelectKBest for each target column and save rankings
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
    
    # Feature selection using SelectKBest
    selector = SelectKBest(score_func=f_regression, k="all")
    selector.fit(X_train, y_train)
    feature_scores = pd.Series(selector.scores_, index=X_train.columns)
    ranked_features = feature_scores.sort_values(ascending=False)
    
    # Save the rankings in a dictionary
    feature_importance_rankings[column] = ranked_features
    
    # Train the full Gaussian Process model as a baseline
    model = GaussianProcessRegressor(kernel=kernel, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # Evaluate and store baseline performance with all features
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_bloodbiomk2_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
        
        # Select the top `n_features` based on initial ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate model with selected features
        model = GaussianProcessRegressor(kernel=kernel, random_state=42)
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Append results for each number of top features
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_bloodbiomk2_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\gaussian_process\_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\fargor\AppData\Lo

C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\gaussian_process\_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\gaussian_process\_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\gaussian_process\_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
  

Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx
